In [2]:
## notebook to remove data where there is overlap between the target and the response

%config Completer.use_jedi = False
import json
import csv
import pandas as pd
import re
from tqdm import tqdm
import math
import random
import string
from tqdm import tqdm
from pprint import pprint
from IPython.display import JSON, display
from collections import Counter, defaultdict
import collections

In [17]:
# First copy data from the Otters repo in the Otters folder

type_split = 'test'
type_domain = "in_domain"
# type_domain = "in_domain"
src_data_ood = pd.read_csv('OTTers/data/'+type_domain+'/'+type_split+'/source.csv')
tgt_data_ood = pd.read_csv('OTTers/data/'+type_domain+'/'+type_split+'/target.csv')
src_data_ood.columns = ['index','source', 'target']
tgt_data_ood.columns = ['index','output']

del tgt_data_ood['index']

frames = [src_data_ood, tgt_data_ood]
data_ood = pd.concat(frames, axis=1, join="inner")

data_ood['domain']=type_domain
# src_data_ood['output'] = tgt_data_ood['output']
data_ood.head()


,index,source,target,output,domain
0,2,my job helpe me teach kids.,education is a passion of mine.,"Yes, I am a teacher too. Education is a passi...",in_domain
1,3,my job helpe me teach kids.,education is a passion of mine.,I admire educators.,in_domain
2,4,my job helpe me teach kids.,education is a passion of mine.,Education has always been a passion of mine as...,in_domain
3,5,my job helpe me teach kids.,education is a passion of mine.,I love helping people learn,in_domain
4,6,my job helpe me teach kids.,education is a passion of mine.,I admire educators.,in_domain


In [18]:
def check_presenceand_remove(target, output):
    targetnopunc = target.translate(str.maketrans('', '', string.punctuation))
    outputnopunc = output.translate(str.maketrans('', '', string.punctuation))
    
    twords = targetnopunc.split()
    owords = outputnopunc.split()
    num_found = 0
    for w in twords:
        if w in owords:
            num_found+=1 
    frac_common = float(num_found)/len(twords)
#     print(frac_common, target, ' --> ', output)
    if frac_common>0.74:
        firstocc = output.lower().find(target.lower()[:10])
        o_afterrem = output[:firstocc].strip()
#         print('\tAfter removal: ', o_afterrem, ' --  ', o_afterrem[-1:])
#         print("{0!r}".format(o_afterrem[-3:]))
        if len(o_afterrem)>0 and o_afterrem[-1]=='.':
#             print('moded')
            return o_afterrem
        else:
            return None
        
    return output

new_data = []
faulty = 0
for index, row in data_ood.iterrows():
    index, source, target, output, domain = row['index'], row['source'], row['target'], row['output'], row.get('domain', '')
    output = output.strip()
    # outputm = check_presenceand_remove(target, output)
    outputm = output

    if outputm is not None:
        new_data.append([index, source, target, outputm, domain])
    else:
        faulty+=1
#     if index>50:        break
        
print(len(new_data), faulty)

with open("simple/"+type_domain+'_'+ type_split+ "_out.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(new_data)

1157 0
